In [2]:
# importamos librerías
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv

# Cargamos variables de entorno
load_dotenv()

True

In [3]:
# endpoint de la API de la agenda de actividades y eventos de Madrid
url = os.getenv("API_URL")

In [ ]:
# Hacemos la petición y comprobamos el estado
response = requests.get(url)
response.status_code

200

In [ ]:
# Convertimos a json
data = response.json()

dict_keys(['@context', '@graph'])

In [7]:
# seleccionamos el elemento del diccionario obtenido que contiene la información de los eventos
eventos = data.get("@graph")

In [8]:
# Obtenemos los nombres de los eventos que están en curso durante las fechas de estancia
from datetime import datetime #importamos librería para transformar fechas
eventos_fechas = {"nombre_evento":[], 
                  "url_evento": [], 
                  "codigo_postal": [], 
                  "direccion": [], 
                  "horario": [],
                  "fecha_inicio": [],
                  "fecha_fin":[], 
                  "organizacion":[]
                  } #diccionario para almacenar los datos *faltan fechas inicio y fin*
#transformamos las fechas de reservas para tener solo la fecha
fecha_entrada = datetime.strptime("2025-03-01", "%Y-%m-%d").date()
fecha_salida = datetime.strptime("2025-03-02", "%Y-%m-%d").date()
# iteramos por cada elemento del diccionario
for evento in eventos:
    # transformamos la fecha para manterner el mismo formato
    fecha_inicio = datetime.strptime(evento["dtstart"], "%Y-%m-%d %H:%M:%S.%f").date()
    fecha_fin = datetime.strptime(evento["dtend"], "%Y-%m-%d %H:%M:%S.%f").date()
    if (fecha_inicio <= fecha_entrada and fecha_fin >= fecha_salida): #si el evento empieza antes o en la fecha de entrada y termina en la fecha de salida o después
        nombre = evento.get("title")
        eventos_fechas["nombre_evento"].append(nombre)
        link = evento.get("link")
        eventos_fechas["url_evento"].append(link)
        direccion_comp = evento.get("address")
        if direccion_comp !=None:
            area = evento.get("address").get("area")
            if area != None:
                codigo_postal=evento.get("address").get("area").get("postal-code")
                direccion = evento.get("address").get("area").get("street-address")
            else:
                codigo_postal= "Dirección no disponible"
                direccion = "Dirección no disponible"
        else: 
            codigo_postal= "Dirección no disponible"
            direccion = "Dirección no disponible"
        eventos_fechas["codigo_postal"].append(codigo_postal)
        eventos_fechas["direccion"].append(direccion)
        hora = evento.get("time")
        eventos_fechas["horario"].append(hora)
        eventos_fechas["fecha_inicio"].append(fecha_inicio)
        eventos_fechas["fecha_fin"].append(fecha_fin)
        organiza = evento.get("organization")
        if organiza != None:
            organizacion = evento.get("organization").get("organization-name")
        else:
            organizacion = "Organización no disponible"
        eventos_fechas["organizacion"].append(organizacion)
#Generamos un dataframe con los eventos
df_eventos = pd.DataFrame(eventos_fechas)
#Guardamos el dataframe en un archivo pickle
df_eventos.to_pickle("../data/data_clean/eventos.pkl")